# Falcon 7B

In [1]:
# Need to check this is running in a hugh RAM environment
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
# Install libraries required for project
!pip install accelerate
!pip install bitsandbytes
!pip install einops
!pip install peft
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00


In [4]:
import bitsandbytes
from peft import PeftConfig, PeftModel
import torch
from transformers import AutoModelForCausalLM, GenerationConfig, AutoTokenizer, BitsAndBytesConfig

In [5]:
model_path="tiiuae/falcon-7b-instruct"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

In [15]:
input_text = "In a list return the sentiment, topic, subtopic, location and the crisis for which this tweet relates: RT @CBCAlerts: Canmore, Alta. declares state of emergency due to flooding  - with some residents being moved to community centre #Alberta"

In [16]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
# Change to required length, too long and may cause OOM issue
max_length = 120
current_length = input_ids.shape[1]

while True:
  # Check if we've reached the length limit
  if current_length >= max_length:
    break

  output = model(next_input)
  next_token_logits = output.logits[:, -1, :]
  next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
  print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

  next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

  current_length += 1

  if next_token_id[0].item() == tokenizer.eos_token_id:
      break



The sentiment of this tweet is 'Emergency'. The topic is 'Flooding'. The subtopic is 'Canmore, Alta.' The location is 'Alberta'. The crisis is related to the flooding in Canmore, Alberta.